In [1]:
#匯入相關所需的模組
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from timeit import default_timer as timer
from keras.preprocessing import image
import numpy as np
import os
from PIL import Image
from keras.models import load_model
import pandas as pd


Using TensorFlow backend.


In [2]:
#將圖像大小調整為128
image_height = 128
image_width = 128

In [3]:
#初始化CNN 建立一個含有6層的卷積層的網路
predator = Sequential()
# 第一層
predator.add(Conv2D(32, (3, 3), activation="relu", input_shape=(image_height, image_width, 3)))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# 添加第二層
predator.add(Conv2D(64, (3, 3), activation="relu"))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# 添加第三層
predator.add(Conv2D(128, (3, 3), activation="relu"))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# 添加第四層
predator.add(Conv2D(256, (3, 3), activation="relu"))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# 添加第五及六層
predator.add(Conv2D(128, (3, 3), activation="relu"))
predator.add(Conv2D(64, (3, 3), activation="relu"))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# 將3D圖像展開成單行陣列
predator.add(Flatten())    
# Full connection
predator.add(Dense(units=32, activation="relu"))
predator.add(Dense(units=20, activation="softmax"))   #輸出層有20個神經元 每個神經元代表一個角色
# 編譯CNN並且展示model架構
predator.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
predator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 256)       295168    
__________

In [4]:
#用ImageDataGenerator將圖片做預處理並且將圖片載入
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory("/home/t107368084/hw2/train/characters-20",target_size = (image_height, image_width),batch_size = 50,class_mode = 'categorical')
test_set = test_datagen.flow_from_directory("/home/t107368084/hw2/test/characters-20",target_size = (image_height, image_width),batch_size = 50,class_mode = 'categorical')

Found 15585 images belonging to 20 classes.
Found 3963 images belonging to 20 classes.


In [5]:
#設定訓練相關數值
predator.fit_generator(training_set,steps_per_epoch = 16,epochs = 100,validation_data = test_set,validation_steps = 4)

Epoch 1/100
16/16 [==============================] - 18s 1s/step - loss: 2.9313 - acc: 0.0700 - val_loss: 3.0246 - val_acc: 0.0500
Epoch 2/100
16/16 [==============================] - 16s 1s/step - loss: 2.8577 - acc: 0.1087 - val_loss: 3.0551 - val_acc: 0.0350
Epoch 3/100
16/16 [==============================] - 16s 984ms/step - loss: 2.8377 - acc: 0.1300 - val_loss: 3.3512 - val_acc: 0.0350
Epoch 4/100
16/16 [==============================] - 16s 1s/step - loss: 2.8176 - acc: 0.1275 - val_loss: 3.2175 - val_acc: 0.0350
Epoch 5/100
16/16 [==============================] - 16s 1s/step - loss: 2.8362 - acc: 0.1475 - val_loss: 3.1355 - val_acc: 0.0350
Epoch 6/100
16/16 [==============================] - 16s 1s/step - loss: 2.8440 - acc: 0.1212 - val_loss: 3.1705 - val_acc: 0.0350
Epoch 7/100
16/16 [==============================] - 16s 1s/step - loss: 2.7883 - acc: 0.1437 - val_loss: 3.1648 - val_acc: 0.0350
Epoch 8/100
16/16 [==============================] - 16s 1s/step - loss: 2.7759 

Epoch 63/100
16/16 [==============================] - 16s 981ms/step - loss: 0.7824 - acc: 0.7575 - val_loss: 0.9953 - val_acc: 0.6950
Epoch 64/100
16/16 [==============================] - 16s 990ms/step - loss: 0.7800 - acc: 0.7638 - val_loss: 1.0331 - val_acc: 0.7000
Epoch 65/100
16/16 [==============================] - 18s 1s/step - loss: 0.7546 - acc: 0.7725 - val_loss: 0.9902 - val_acc: 0.7000
Epoch 66/100
16/16 [==============================] - 17s 1s/step - loss: 0.7154 - acc: 0.7763 - val_loss: 1.1112 - val_acc: 0.6800
Epoch 67/100
16/16 [==============================] - 16s 979ms/step - loss: 0.6966 - acc: 0.8000 - val_loss: 0.9532 - val_acc: 0.7050
Epoch 68/100
16/16 [==============================] - 16s 986ms/step - loss: 0.7022 - acc: 0.8012 - val_loss: 0.9454 - val_acc: 0.7200
Epoch 69/100
16/16 [==============================] - 16s 975ms/step - loss: 0.6461 - acc: 0.8043 - val_loss: 1.0579 - val_acc: 0.6950
Epoch 70/100
16/16 [==============================] - 16s 982

In [6]:
#載入test data並辨識
result = []
path = '/home/t107368084/hw2/valid/test'
files = os.listdir(path)

for file in files:
    test_image = image.load_img('/home/t107368084/hw2/valid/test/'+file, target_size=(image_height, image_width))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    pred = predator.predict_on_batch(test_image)
    result.append(pred)

result = np.asarray(result)

In [7]:
#將結果表格化並輸出成CSV檔
index = files
predictions = result[:, [0]][:,0]
df = pd.DataFrame(index=index)
#以下是我們本次作業要辨認的20位人物
df['abraham_grampa_simpson'] = predictions[:,0]
df['apu_nahasapeemapetilon'] = predictions[:,1]
df['bart_simpson'] = predictions[:,2]
df['charles_montgomery_burns'] = predictions[:,3]
df['chief_wiggum'] = predictions[:,4]
df['comic_book_guy'] = predictions[:,5]
df['edna_krabappel'] = predictions[:,6]
df['homer_simpson'] = predictions[:,7]
df['kent_brockman'] = predictions[:,8]
df['krusty_the_clown'] = predictions[:,9]
df['lenny_leonard'] = predictions[:,10]
df['lisa_simpson'] = predictions[:,11]
df['marge_simpson'] = predictions[:,12]
df['mayor_quimby'] = predictions[:,13]
df['milhouse_van_houten'] = predictions[:,14]
df['moe_szyslak'] = predictions[:,15]
df['ned_flanders'] = predictions[:,16]
df['nelson_muntz'] = predictions[:,17]
df['principal_skinner'] = predictions[:,18]
df['sideshow_bob'] = predictions[:,19]


df = df.astype(int)

df.to_csv('/home/t107368084/hw2/predictions.csv')

In [8]:
#儲存模型

predator.save_weights('/home/t107368084/hw2/simpsons.hdf5')